# VRDL HW3

The code for the assignment of Selected Topics in Visual Recognition using Deep Learning, NCTU, in fall 2020.


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f3536e65-4dc1-e437-e1f3-1234cffd10f1)


## Mount google drive

In [2]:
# If need to mount google drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


## Download dataset

In [3]:
!gdown -O ./train_images.zip 'https://drive.google.com/uc?export=download&id=1VwaQyKkVa6i2F9pCQiBZrWoWWfSQJW_t'
!gdown -O ./pascal_train.json 'https://drive.google.com/uc?export=download&id=1HwouAXDLbovGf7sBfhgboDcU9VqgNf6s'

!gdown -O ./test_images.zip 'https://drive.google.com/uc?export=download&id=18cKfSD8msgB3GVyGdHWK6TnByyb-zq5X'
!gdown -O ./test.json 'https://drive.google.com/uc?export=download&id=1VAhbaxLH7mjoPcJpoji5WijEoJy87wkk'

Downloading...
From: https://drive.google.com/uc?export=download&id=1VwaQyKkVa6i2F9pCQiBZrWoWWfSQJW_t
To: /content/train_images.zip
149MB [00:02, 65.6MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1HwouAXDLbovGf7sBfhgboDcU9VqgNf6s
To: /content/pascal_train.json
10.4MB [00:00, 63.2MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=18cKfSD8msgB3GVyGdHWK6TnByyb-zq5X
To: /content/test_images.zip
12.1MB [00:00, 73.4MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1VAhbaxLH7mjoPcJpoji5WijEoJy87wkk
To: /content/test.json
100% 8.57k/8.57k [00:00<00:00, 14.7MB/s]


In [4]:
!unzip ./train_images.zip -d ./data/
!unzip ./test_images.zip -d ./data/

Archive:  ./train_images.zip
   creating: ./data/train_images/
  inflating: ./data/train_images/2007_000033.jpg  
  inflating: ./data/train_images/2007_000042.jpg  
  inflating: ./data/train_images/2007_000061.jpg  
  inflating: ./data/train_images/2007_000123.jpg  
  inflating: ./data/train_images/2007_000129.jpg  
  inflating: ./data/train_images/2007_000175.jpg  
  inflating: ./data/train_images/2007_000187.jpg  
  inflating: ./data/train_images/2007_000323.jpg  
  inflating: ./data/train_images/2007_000332.jpg  
  inflating: ./data/train_images/2007_000346.jpg  
  inflating: ./data/train_images/2007_000452.jpg  
  inflating: ./data/train_images/2007_000464.jpg  
  inflating: ./data/train_images/2007_000491.jpg  
  inflating: ./data/train_images/2007_000529.jpg  
  inflating: ./data/train_images/2007_000559.jpg  
  inflating: ./data/train_images/2007_000572.jpg  
  inflating: ./data/train_images/2007_000636.jpg  
  inflating: ./data/train_images/2007_000661.jpg  
  inflating: ./data

In [5]:
!mkdir ./data/annotations
!mv ./pascal_train.json ./data/annotations/pascal_train.json
!mv ./test.json ./data/annotations/test.json

In [6]:
!rm -rf ./sample_data
!rm -rf ./train_images.zip
!rm -rf ./test_images.zip

## Install MMDetection

In [7]:
# Install dependencies
!pip install -U torch==1.6+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Install the latest mmcv
!pip install mmcv-full==latest+torch1.6.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

# Install albumentations
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements/build.txt
!pip install -e .

"""
# Install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0
"""

# Check Pytorch installation
import torch, torchvision

print(torch.__version__, torch.cuda.is_available())


# Check MMDetection installation
import mmdet

print(mmdet.__version__)


# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print(get_compiling_cuda_version())
print(get_compiler_version())

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 25kB/s 
     |████████████████████████████████| 5.9MB 37.5MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 22.4MB 143kB/s 
     |████████████████████████████████| 194kB 7.9MB/s 
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-nfgfd76c
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-nfgfd76c
     |████████████████████████████████| 952kB 8.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.5.2-cp36-none-any.whl size=72232 sha256=111c2337f190b5c3f

In [8]:
%cd ..

/content


## Split dataset

In [9]:
from tinyVOC.utils import split_ann

In [10]:
_ = split_ann(
    './data/annotations/pascal_train.json',
    './data/annotations/instances_train.json',
    './data/annotations/instances_val.json')

## Main

In [11]:
# Please change the work_dir to your own work directory
work_dir = './work_dirs/mask_rcnn_r50_fpn_40e_tinyVOC'
config_file = './mask_rcnn_r50_fpn_40e_tinyVOC.py'

### Train

In [12]:
train_config = './mask_rcnn_r50_fpn_40e_tinyVOC_train.py'

In [13]:
# You can modify the config for train here
from mmcv import Config

cfg = Config.fromfile(config_file)
#cfg.data.test.ann_file = './data/annotations/instances_val.json'
#cfg.data.test.img_prefix = './data/train_images/'
#cfg.data.samples_per_gpu //= 2
#cfg.optimizer.lr /= 2
#cfg.workflow = [('train', 1)]

with open(train_config, 'w') as f:
    f.write(cfg.pretty_text)

In [ ]:
!python ./mmdetection/tools/train.py {train_config} --work-dir {work_dir}

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {work_dir}/tf_logs

### Eval

In [16]:
# Download the checkpoint
!gdown -O {work_dir}/epoch_40.pth 'https://drive.google.com/uc?export=download&id=12kAEsDYkPPB6lKOg567rLqXIr6PiCs-b'

Downloading...
From: https://drive.google.com/uc?export=download&id=12kAEsDYkPPB6lKOg567rLqXIr6PiCs-b
To: /content/work_dirs/mask_rcnn_r50_fpn_40e_tinyVOC/epoch_40.pth
352MB [00:03, 108MB/s] 


In [17]:
eval_config = './eval_config.py'
checkpoint = 'epoch_40.pth'

In [18]:
from mmcv import Config

cfg = Config.fromfile(config_file)
cfg.data.test.ann_file = './data/annotations/instances_val.json'
cfg.data.test.img_prefix = './data/train_images/'

with open(eval_config, 'w') as f:
    f.write(cfg.pretty_text)

In [19]:
!python ./mmdetection/tools/test.py {eval_config} {work_dir}/{checkpoint} --eval segm

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
[>>] 269/269, 3.0 task/s, elapsed: 89s, ETA:     0s
Evaluating segm...
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.53s).
Accumulating evaluation results...
DONE (t=0.18s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.311
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.552
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.288
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.101
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.238
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.437
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.415
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.4

### Test

In [20]:
test_config = config_file
result_file = './result'

In [21]:
!python ./mmdetection/tools/test.py {test_config} {work_dir}/{checkpoint} --format-only --options jsonfile_prefix={work_dir}/{result_file}

./mmdetection/tools/test.py:92: UserWarning: --options is deprecated in favor of --eval-options
  warnings.warn('--options is deprecated in favor of --eval-options')
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[>>] 100/100, 2.9 task/s, elapsed: 34s, ETA:     0s